In [27]:
import pandas as pd
import numpy as np
import json

In [28]:
#importing data (pre1790)
pre1790 = pd.read_csv('prepost_data_aggregations/statewise_debt_aggregation_Pre1790.csv')
pre1790.columns = ['state', 'Loan Office Certificates Specie Value','State Liquidated Debt Dollars','Pierce Certificates Value']

In [29]:
#importing data (post1790)
post1790 = pd.read_csv('prepost_data_aggregations/statewise_debt_aggregation_Post1790.csv', index_col = 0)
post1790_ds = pd.read_csv('prepost_data_aggregations/statewise_debt_aggregation_Post1790_debtstate.csv', index_col = 0)
post1790_rep = pd.read_csv('prepost_data_aggregations/statewise_debt_aggregation_Post1790_rep.csv', index_col = 0)

In [44]:
state_pop = pd.read_csv('../Data/statepop.csv')[['State', 'Total Pop', 'Slave Pop']]

In [45]:
state_pop['White Pop'] = state_pop['Total Pop'] - state_pop['Slave Pop']
state_pop.dropna(inplace = True)
state_pop.c

,State,Total Pop,Slave Pop,White Pop
0,Maine,49133.0,458.0,48675.0
1,New Hampshire,87802.0,541.0,87261.0
2,Vermont,47620.0,50.0,47570.0
3,Massachusetts,268627.0,4822.0,263805.0
4,Rhode Island,52946.0,2671.0,50275.0
5,Connecticut,206701.0,5885.0,200816.0
6,New York,210541.0,21054.0,189487.0
7,New Jersey,139627.0,10460.0,129167.0
8,Pennsylvania,327305.0,7855.0,319450.0
9,Delaware,45385.0,2996.0,42389.0


In [31]:
with open('state_abbrev.json') as file:
    state_abbrev = json.load(file)

In [32]:
state_pop['state'] = state_pop['state'].apply(lambda x: state_abbrev[x])

In [33]:
#pre 1790 and post1790 asset totals
pre1790cols = ['Loan Office Certificates Specie Value', 'State Liquidated Debt Dollars', 'Pierce Certificates Value']
post1790cols = ['ASD Total', 'CD Total']
pre1790['pre1790 total'] = pre1790[pre1790cols].sum(axis = 1)
post1790['post1790 total'] = post1790[post1790cols].sum(axis = 1)
post1790_ds['post1790 total'] = post1790_ds[post1790cols].sum(axis = 1)
post1790_rep['post1790 total'] = post1790_rep[post1790cols].sum(axis = 1)
pre1790_tot = pre1790['pre1790 total'].sum()
post1790_tot = post1790['post1790 total'].sum()
print("pre1790: {:.2f}, post1790: {:.2f}, ratio: {:.2f}".format(pre1790_tot, post1790_tot, post1790_tot/pre1790_tot))

pre1790: 26860611.73, post1790: 14476118.95, ratio: 0.54


In [34]:
def calculateMetrics(df):
    #aggregates
    df = df.fillna(0)
    df = pd.merge(df, state_pop, on = 'state', how = 'outer')
    
    df['pre1790 pct'] = df['pre1790 total']/pre1790_tot * 100
    df['post1790 pct'] = df['post1790 total']/post1790_tot * 100
    df['total asset dif'] = df['post1790 total'] - df['pre1790 total']
    #reweighting of post1790 assets
    df['total asset dif (adj)'] = df['post1790 total']*pre1790_tot/post1790_tot - all_debts_ds['pre1790 total']
    df['total asset pct dif'] = (df['post1790 pct'] - df['pre1790 pct'])
    
    df['Loan Office pct'] =  df['Loan Office Certificates Specie Value']/df['Loan Office Certificates Specie Value'].sum()
    df['State Liquidated pct'] =  df['State Liquidated Debt Dollars']/df['State Liquidated Debt Dollars'].sum()
    df['Pierce pct'] =  df['Pierce Certificates Value']/df['Pierce Certificates Value'].sum()
    df['CD pct'] =  df['CD Total']/df[ 'CD Total'].sum()
    df['ASD pct'] =  df['ASD Total']/df['ASD Total'].sum()
    
    df['pre1790 total (pop adj)'] = df['pre1790 total']/df['pop']
    df['post1790 total (pop adj)'] = df['post1790 total']/df['pop']
    
    df['pre1790 pct (pop adj)'] = df['pre1790 total (pop adj)']/df['pre1790 total (pop adj)'].sum() * 100
    df['post1790 pct (pop adj)'] = df['post1790 total (pop adj)']/df['post1790 total (pop adj)'].sum() * 100 
    df['total asset dif (pop adj)'] = df['post1790 total (pop adj)'] - df['pre1790 total (pop adj)']
    df['total asset pct dif (pop adj)'] =  df['post1790 pct (pop adj)'] - df['pre1790 pct (pop adj)']
    return df.round(2)

In [35]:
#default

In [36]:
all_debts = pd.merge(pre1790[['state', 'pre1790 total']], post1790[['state', 'post1790 total']], 
                     on = 'state', how = 'outer').round(2)
#all_debts

In [37]:
post1790_ds.columns = ['state', 'ASD Total', 'CD Total', 'post1790 total']
all_debts_ds = pd.merge(pre1790, post1790_ds, 
                        on = 'state', how = 'outer').round(2)
all_debts_ds = calculateMetrics(all_debts_ds)
all_debts_ds

,state,Loan Office Certificates Specie Value,State Liquidated Debt Dollars,Pierce Certificates Value,pre1790 total,ASD Total,CD Total,post1790 total,pop,pre1790 pct,...,State Liquidated pct,Pierce pct,CD pct,ASD pct,pre1790 total (pop adj),post1790 total (pop adj),pre1790 pct (pop adj),post1790 pct (pop adj),total asset dif (pop adj),total asset pct dif (pop adj)
0,PA,3778570.27,2257929.43,1147498.52,7183998.22,0.00,2952380.76,2952380.76,240057.0,26.75,...,0.26,0.14,0.26,0.00,29.93,12.30,18.64,13.25,-17.63,-5.40
1,MA,2387481.08,457823.49,2217598.90,5062903.47,0.00,0.00,0.00,235308.0,18.85,...,0.05,0.27,0.00,0.00,21.52,0.00,13.40,0.00,-21.52,-13.40
2,CT,1275499.48,135432.22,1137988.73,2548920.43,366794.04,1389315.78,1756109.82,183881.0,9.49,...,0.02,0.14,0.12,0.11,13.86,9.55,8.64,10.29,-4.31,1.65
3,NY,922005.66,3649822.51,774934.83,5346763.00,1408932.24,1926464.36,3335396.60,162920.0,19.91,...,0.42,0.09,0.17,0.44,32.82,20.47,20.45,22.05,-12.35,1.60
4,NJ,651204.36,1768007.24,513348.62,2932560.22,0.00,0.00,0.00,117431.0,10.92,...,0.21,0.06,0.00,0.00,24.97,0.00,15.56,0.00,-24.97,-15.56
5,MD,404704.96,0.00,550014.80,954719.76,143823.61,1447729.90,1591553.51,202599.0,3.55,...,0.00,0.07,0.13,0.04,4.71,7.86,2.94,8.46,3.14,5.52
6,NH,300171.07,13826.44,380285.90,694283.41,0.00,307782.89,307782.89,62396.0,2.58,...,0.00,0.05,0.03,0.00,11.13,4.93,6.93,5.31,-6.19,-1.62
7,VA,235249.60,0.00,653914.72,889164.32,264976.95,859264.17,1124241.12,447016.0,3.31,...,0.00,0.08,0.08,0.08,1.99,2.51,1.24,2.71,0.53,1.47
8,DE,66387.12,136305.94,130868.22,333561.28,0.00,0.00,0.00,35496.0,1.24,...,0.02,0.02,0.00,0.00,9.40,0.00,5.85,0.00,-9.40,-5.85
9,RI,0.00,183000.46,81210.76,264211.22,64363.22,814182.09,878545.31,58196.0,0.98,...,0.02,0.01,0.07,0.02,4.54,15.10,2.83,16.26,10.56,13.43


In [38]:
post1790_rep.loc[post1790_rep[post1790_rep['state'] == 'NY loan office'].index, 'state'] = 'NY'

In [39]:
post1790_rep.loc[post1790_rep[post1790_rep['state'] == 'NY loan office'].index, 'state'] = 'NY'
post1790_rep = post1790_rep.groupby('state')[['ASD Total','CD Total', 'post1790 total']].sum().reset_index()
all_debts_rep = pd.merge(pre1790, post1790_rep, 
                        on = 'state', how = 'outer').round(2)
all_debts_rep = calculateMetrics(all_debts_rep)
all_debts_rep

,state,Loan Office Certificates Specie Value,State Liquidated Debt Dollars,Pierce Certificates Value,pre1790 total,ASD Total,CD Total,post1790 total,pop,pre1790 pct,...,State Liquidated pct,Pierce pct,CD pct,ASD pct,pre1790 total (pop adj),post1790 total (pop adj),pre1790 pct (pop adj),post1790 pct (pop adj),total asset dif (pop adj),total asset pct dif (pop adj)
0,PA,3778570.27,2257929.43,1147498.52,7183998.22,129119.33,2843338.75,2972458.08,240057.0,26.75,...,0.26,0.14,0.25,0.04,29.93,12.38,18.64,13.63,-17.54,-5.02
1,MA,2387481.08,457823.49,2217598.90,5062903.47,18523.71,135257.04,153780.75,235308.0,18.85,...,0.05,0.27,0.01,0.01,21.52,0.65,13.40,0.72,-20.86,-12.68
2,CT,1275499.48,135432.22,1137988.73,2548920.43,282429.55,1346300.72,1628730.27,183881.0,9.49,...,0.02,0.14,0.12,0.09,13.86,8.86,8.64,9.75,-5.00,1.11
3,NY,922005.66,3649822.51,774934.83,5346763.00,1762877.57,1952449.73,3715327.30,162920.0,19.91,...,0.42,0.09,0.17,0.55,32.82,22.80,20.45,25.09,-10.01,4.65
4,NJ,651204.36,1768007.24,513348.62,2932560.22,1607.80,43128.01,44735.81,117431.0,10.92,...,0.21,0.06,0.00,0.00,24.97,0.38,15.56,0.42,-24.59,-15.14
5,MD,404704.96,0.00,550014.80,954719.76,141713.62,1436570.68,1578284.30,202599.0,3.55,...,0.00,0.07,0.13,0.04,4.71,7.79,2.94,8.57,3.08,5.64
6,NH,300171.07,13826.44,380285.90,694283.41,544.07,272954.78,273498.85,62396.0,2.58,...,0.00,0.05,0.02,0.00,11.13,4.38,6.93,4.82,-6.74,-2.11
7,VA,235249.60,0.00,653914.72,889164.32,275087.60,886516.08,1161603.68,447016.0,3.31,...,0.00,0.08,0.08,0.09,1.99,2.60,1.24,2.86,0.61,1.62
8,DE,66387.12,136305.94,130868.22,333561.28,0.00,12781.59,12781.59,35496.0,1.24,...,0.02,0.02,0.00,0.00,9.40,0.36,5.85,0.40,-9.04,-5.46
9,RI,0.00,183000.46,81210.76,264211.22,62807.17,766286.96,829094.13,58196.0,0.98,...,0.02,0.01,0.07,0.02,4.54,14.25,2.83,15.68,9.71,12.85


In [40]:
all_debts_ds.to_csv('statewise_all_assets_ds.csv')

In [41]:
all_debts_rep.to_csv('statewise_all_assets_rep.csv')